In [1]:
import tensorflow as tf

seq2seq 模型最开始应用于机器翻译。

机器翻译
早期的机器翻译的思路十分简单，通过设置大量的翻译规则，构建一个大型的双语对照表，来将源语言翻译成目标语言。

后来1990-2010s, 研究者梦开发了更先进复杂的机器翻译技术，（统计机器翻译）Statistical Machine Translation, SMT.的主要原理是从大量的数据中学习一个概率模型p(y|x), 其中x是源语言，y是目标语言。翻译的时候，需要通过求argmaxyP(y|x) 就行

y和x 都是一个句子。

$$P(y|x)=\frac{P(y)P(x|y)}{P(x)}$$

那么
$$argmax_yP(y|x)=argmax_y\frac{P(y)P(x|y)}{P(x)}=argmax_yP(y)P(x|y)$$

P(y)是一个语言模型，而P(x|y)则被称为翻译模型。LM通过目标语料库进行训练，而TM通过平行语料进行训练。

在学习完LM和TM这两个模型之后，需要使用模型，即寻找最佳翻译，y的过程。这个过程也被称为decoding

decoding的做法是通过CRF或HMM的解码。统计机器翻译，虽然比较强但是存在一些问题
- SMT的门槛比较高，表现比较好的模型，通常比较复杂
- SMT模型需要庞大的人力去维护

神经机器翻译（NMT）
NMT的优势在于将大量的人工操作进行省略，下面是SMT和NMT的对比
![title](img/SMT-vs-NMT.png)
NMT 使用网络结构是sequence-to-sequence, 即seq2seq。 同时也被称为encoder-decoder结构。


![title](img/seq2seq_1.png)
根据图中结构，seq2seq中的Encoder读取输入文本，也就是源语言文本。encoder的作用是将输入文本转化成向量，Decoder的作用是将向量转化为输出文本，即目标语言文本。Encoder的行为是编码，而Decoder的行为是解码。

对于其中的具体结构，如下所示

在Encoder短，将source通过Embedding层，将文本转化为词向量，然后经过一个RNN\LSTM\GRU等RNN类型网络层，输出背景向量（context vector）

在Decoder端， 输入的是背景向量以及目标语言的词向量。目标语言的第一个输入是<start>, 每一步根据当前正确的输入词以及上一步的隐状态来预处下一步的输出词。
    
   
 对于预测的时候，Encoder端和训练时相同，而Decoder端部分需要Context vector 以及 初始输入"<start>"。然后后面的RNN结构的网络层的每一次的输入是上一个单元的隐藏层，以及预测的本次的单词的词向量。直到预测的单词为"<end>"或者序列长度达到指定值为止。
    

In [2]:
with open("cmn.txt", "r", encoding="utf-8") as f:
    data = f.read()

In [5]:
# 预处理数据
en = []
cn = []
for x in data.split("\n"):
    if len(x.split("\t")) < 2:
        continue
    ei, ci = x.split("\t")
    en.append(ei)
    cn.append(ci)
    
    